# Lots de tâches
Le calcul haute-performance consiste non seulement au calcul parallèle par tâche (***parallélisme des tâches***),
mais aussi au calcul de données en parallèle dans plusieurs tâches et/ou processus en simultané (***parallélisme de données***).
Ce chapitre vous donnera les outils nécessaires pour gérer un grand nombre de tâches
lorsque le projet de recherche requiert plusieurs centaines de résultats.

## GNU Parallel
La commande `parallel` de
[GNU Parallel](https://docs.alliancecan.ca/wiki/GNU_Parallel/fr)
permet d'utiliser pleinement les ressources locales d'un noeud de
calcul, et ce, en gérant l'exécution d'une **longue liste de tâches
de *petite* taille**.
C'est un peu comme l'ordonnanceur Slurm, mais à plus petite échelle et
en gérant des processus au lieu de scripts de tâche.

![Fonctionnement de GNU Parallel](images/gnu-parallel.svg)

* [Documentation officielle](https://www.gnu.org/software/parallel/parallel.html)
* [Tutoriel](https://www.gnu.org/software/parallel/parallel_tutorial.html)

### Pourquoi pas Slurm?
OK, mais pourquoi ne pas tout simplement soumettre
**des centaines de tâches à Slurm**?
* À tout moment, Slurm **limite chaque usager à 1000 tâches**
  au total dans `squeue` (*pending* + *running*)
* Certains calculs sont tellement **courts (< 5 minutes)** que le
  démarrage et la fin de la tâche compteraient pour un pourcentage
  significatif du temps réel utilisé, ce qui diminue leur efficacité

Les avantages de GNU Parallel à considérer :
* Nous **évite d'utiliser une boucle** soumettant des centaines de
  scripts similaires, ce qui, dans bien des cas, facilite
  l'exécution de centaines de cas de calcul semblables
* Le nombre de **processeurs disponibles limite** automatiquement le
  nombre de cas de calcul exécutés en simultané
  * Dans le cas de calculs parallèles, c'est possible de spécifier
    le nombre de cas en simultané
* GNU Parallel peut
  [reprendre la séquence des cas de calcul](https://docs.alliancecan.ca/wiki/GNU_Parallel/fr#Suivi_des_commandes_ex.C3.A9cut.C3.A9es_ou_des_commandes_ayant_.C3.A9chou.C3.A9.3B_fonctionnalit.C3.A9s_de_red.C3.A9marrage)
  en situation de fin hâtive de la tâche Slurm

### Syntaxe de la commande GNU Parallel
Les éléments de base de la commande `parallel` :
```Bash
parallel options gabarit_de_commande ::: liste de valeurs
```

Voir la page de manuel :
```Bash
man parallel  # Appuyez sur Q pour quitter
```

### Modes d'utilisation
#### Une seule séquence de paramètres
Le paramètre changeant est donné via une paire d'`{}` :
```Bash
parallel echo fichier{}.txt ::: 1 2 3 4
# parallel --citation  # S'engager à citer les développeurs
```

On peut réécrire la première commande en utilisant l'expansion des
accolades Bash `{a..b}` :
```Bash
parallel echo fichier{}.txt ::: {1..4}
parallel echo fichier{}.txt ::: {01..10}
```

#### Combinaisons de paramètres
**a)** Lorsqu'il y a **plusieurs séquences de paramètres à combiner**,
on peut utiliser des paires d'accolades numérotées telles que
`{1}`, `{2}`, etc. :
```Bash
parallel echo fichier{1}{2}.txt ::: {01..10} ::: a b
```

**b)** Dans le cas où on retrouve les **combinaisons de paramètres
dans un fichier texte** :
```Bash
cat scripts/param.txt
```

La commande `parallel` aura `-C ' '` pour spécifier le séparateur de
paramètres dans `param.txt`, ainsi que l'argument `::::` pour
spécifier ensuite ce nom de fichier :
```Bash
# parallel -C ' ' echo '$(({1}*{2}))' :::: scripts/param.txt
cat scripts/prll-exec-param.sh
sbatch scripts/prll-exec-param.sh
```

**c)** Si on préfère valider la **liste des commandes dans un
fichier texte** avant leur exécution sur un noeud de calcul :
```Bash
cat scripts/cmd.txt
```

Le script de tâche aura une commande `parallel` simplifiée :
```Bash
# parallel < scripts/cmd.txt
cat scripts/prll-exec-cmd.sh
sbatch scripts/prll-exec-cmd.sh
```

#### Nombre limité de cas en parallèles
Le paramètre `--jobs` permet de forcer une limite sur le nombre de
processus lancés à la fois. Par exemple, 8 cas avec 2 processus
en simultané :
```Bash
parallel --jobs 2 'echo {} && sleep 3' ::: {1..8}
```

### **Exercice** - Aligner des séquences d'ADN
Dans le dossier `donnees/`, vous devriez avoir plusieurs
fichiers Fasta (`*.fa`) qui ont été créés au premier
chapitre via le script de tâche `scripts/blastn-gen-seq.sh`.
**Si ce n'est pas le cas**, faites :
```Bash
sbatch scripts/blastn-gen-seq.sh
```

On devrait y retrouver :
* pour chaque espèce fictive A, B, C et D,
  un fichier de séquences d'ADN "connues"
  * Ces fichiers sont convertis en bases de données Blast
* pour 16 espèces "inconnues" K à Z, des séquences d'ADN à aligner
  sur les séquences "connues" des espèces A, B, C et D

On cherche donc à calculer l'alignement de toutes les
combinaisons `{A,B,C,D}` x `{K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z}`,
ce qui donne 64 combinaisons.
Le script de tâche suivant utilise GNU Parallel pour calculer les
différents cas avec 4 processeurs :
```Bash
cat scripts/blastn-parallel.sh
sbatch scripts/blastn-parallel.sh
```

Pour surveiller l'état de la tâche :
```Bash
squeue -u $USER
ls donnees/res_prll/
```
À la fin de la tâche, il est bon d'analyser les ressources utilisées :
```
seff <jobid>
```

### Autres outils
* GLOST
  [pour des calculs séquentiels seulement](https://docs.alliancecan.ca/wiki/GLOST/fr)

* META-Farm
  [pour le meilleur de GNU Parallel et GLOST](https://docs.alliancecan.ca/wiki/META-Farm/fr)

Alors que les précédents outils s'utilisent bien avec un lot de
calculs séquentiels ou parallèles de petite taille (16 processeurs
ou moins), **ils ne sont pas** vraiment **appropriés pour**
un lot de **longs calculs parallèles de plus grande taille**
(plus de 16 processeurs par calcul) :
1. on veut éviter les longues tâches qui dépassent trois (3) jours et
1. on veut réduire le risque de subir une défaillance matérielle.

C'est pourquoi, dans certains cas, il vaut
mieux utiliser les vecteurs de tâches.

## Vecteurs de tâches
Dans le cas où un même programme doit être exécuté avec différentes
combinaisons de paramètres, il y a moyen de soumettre un seul
[vecteur de tâches](https://docs.alliancecan.ca/wiki/Job_arrays/fr)
et de coder le script de tâche de telle sorte que les paramètres
seront déterminés **en fonction d'un indice unique** du vecteur de tâches.

![Fonctionnement des vecteurs de tâches](images/vecteur-taches.svg)

**Pour soumettre un vecteur de tâches** à l'ordonnanceur
Slurm, **on doit ajouter l'option** `--array=<indices>`
à l'entête `#SBATCH` du script de tâche.
Voir [quelques exemples ici](https://docs.alliancecan.ca/wiki/Job_arrays/fr).

L'identifiant d'une tâche Slurm dans un vecteur de tâches contient :
* L'identifiant du vecteur de tâches
* Caractère de soulignement (`_`)
* L'indice unique associé à la tâche

**Par exemple :** `25249551_15`

Dans le script de tâche, la **variable d'environnement**
`$SLURM_ARRAY_TASK_ID` peut être utilisée pour retrouver la valeur
actuelle de l'indice unique associé à la tâche en cours d'exécution.
Il s'agit d'un **entier parmi** les `<indices>`.

La variable `$SLURM_ARRAY_TASK_ID` peut être utilisée de différentes
façons. L'exemple ci-dessous utilise `$N`, mais
`$SLURM_ARRAY_TASK_ID` fonctionne de la même manière :
```Bash
export N=71  # Requis uniquement pour cet exemple

echo fichier.$N
echo répertoire-$N
```

![Entier à coordonnées 2D](images/n2r_c.svg)
```Bash
PARAM_R=$((N / 12))  # Division entière
PARAM_C=$((N % 12))  # Modulo (reste de division)
echo $PARAM_R $PARAM_C

head -n $((PARAM_R + 1)) scripts/param.txt | tail -1
```

### **Exercice** - Vecteur de tâches
Lancez le vecteur de tâches :
```Bash
cat scripts/blastn-array.sh
sbatch scripts/blastn-array.sh

squeue -u $USER
```

Après l'exécution des quatre tâches, inspectez les résultats :
```Bash
ls slurm-*_*.out
ls -l donnees/res_array/
```

* **Corrigez le script de tâche** pour que les 16 inconnues K à Z
  soient traitées, avec une limite de **quatre** tâches à la fois.
* Relancez le vecteur de tâches pour tester la correction.
* Utilisez la commande `seff` pour étudier une des 16 tâches.

## Points à retenir
* **GNU Parallel**
  [pour lancer de multiples combinaisons de paramètres](https://docs.alliancecan.ca/wiki/GNU_Parallel/fr)

```Bash
parallel 'gabarit_cmd({1})' ::: valeurs1
parallel 'gabarit_cmd({1}, {2})' ::: valeurs1 ::: valeurs2
parallel -C <sep> 'gabarit_cmd({1}, {2})' :::: paires_param.txt
parallel --jobs 'N_cas_par_noeud' < liste_commandes.txt
```

* **Vecteurs de tâches**
  [pour lancer une série de longues ou de larges tâches](https://docs.alliancecan.ca/wiki/Job_arrays/fr)

```Bash
# $SLURM_ARRAY_TASK_ID aura une seule valeur dans ...
sbatch --array=0-7      # [0, 7]
sbatch --array=1,3,5,7  # {1,3,5,7}
sbatch --array=1-7:3    # {1,4,7}
sbatch --array=0-99%10  # [0, 99], mais 10 tâches à la fois
```